# Lab Exercise
## De-Duping Data

-sandbox
##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Instructions

In this exercise, we're doing ETL on a file we've received from some customer. That file contains data about people, including:

* first, middle and last names
* gender
* birth date
* Social Security number
* salary

But, as is unfortunately common in data we get from this customer, the file contains some duplicate records. Worse:

* In some of the records, the names are mixed case (e.g., "Carol"), while in others, they are uppercase (e.g., "CAROL"). 
* The Social Security numbers aren't consistent, either. Some of them are hyphenated (e.g., "992-83-4829"), while others are missing hyphens ("992834829").

The name fields are guaranteed to match, if you disregard character case, and the birth dates will also match. (The salaries will match, as well,
and the Social Security Numbers *would* match, if they were somehow put in the same format).

Your job is to remove the duplicate records. The specific requirements of your job are:

* Remove duplicates. It doesn't matter which record you keep; it only matters that you keep one of them.
* Preserve the data format of the columns. For example, if you write the first name column in all lower-case, you haven't met this requirement.
* Write the result as a Parquet file, as designated by *destFile*.
* The final Parquet "file" must contain 8 part files (8 files ending in ".parquet").

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** The initial dataset contains 103,000 records.<br/>
The de-duplicated result haves 100,000 records.

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Getting Started

Run the following cell to configure our "classroom."

In [0]:
%run "./Includes/Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from wasbs://training@dbtraincentralindia.blob.core.windows.net/

Created user-specific database

Using the database farzanam24_gmail_com_db .

All done!

In [0]:
%run "./Includes/Initialize-Labs"

Initializing lab environment: 
 Declared clearYourResults( passedOnly:Boolean=true ) 
 Declared validateYourSchema( what:String, df:DataFrame, expColumnName:String, expColumnType:String ) 
 Declared validateYourAnswer( what:String, expectedHash:Int, answer:Any ) 
 Declared summarizeYourResults() 
 Declared logYourTest( path:String, name:String, value:Double ) 
 Declared loadYourTestResults( path:String ) returns DataFrame 
 Declared loadYourTestMap( path:String ) returns Map[String,Double]

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Hints

* Use the <a href="http://spark.apache.org/docs/latest/api/python/index.html" target="_blank">API docs</a>. Specifically, you might find 
  <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame" target="_blank">DataFrame</a> and
  <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions" target="_blank">functions</a> to be helpful.
* It's helpful to look at the file first, so you can check the format. `dbutils.fs.head()` (or just `%fs head`) is a big help here.

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
partitions = 8

# Make sure wide operations don't repartition to 200
spark.conf.set("spark.sql.shuffle.partitions", str(partitions))

(source, sasEntity, sasToken) = getAzureDataSource()
spark.conf.set(sasEntity, sasToken)

sourceFile = source + "/dataframes/people-with-dups.txt"
destFile = userhome + "/people.parquet"
dbutils.fs.ls(sourceFile)
print(dbutils.fs.head(sourceFile))

# In case it already exists
#dbutils.fs.rm(destFile, True)

[Truncated to first 65536 bytes]
firstName:middleName:lastName:gender:birthDate:salary:ssn
Emanuel:Wallace:Panton:M:1988-03-04:101255:935-90-7627
Eloisa:Rubye:Cayouette:F:2000-06-20:204031:935-89-9009
Cathi:Svetlana:Prins:F:2012-12-22:35895:959-30-7957
Mitchel:Andres:Mozdzierz:M:1966-05-06:55108:989-27-8093
Angla:Melba:Hartzheim:F:1938-07-26:13199:935-27-4276
Rachel:Marlin:Borremans:F:1923-02-23:67070:996-41-8616
Catarina:Phylicia:Dominic:F:1969-09-29:201021:999-84-8888
Antione:Randy:Hamacher:M:2004-03-05:271486:917-96-3554
Madaline:Shawanda:Piszczek:F:1996-03-17:183944:963-87-9974
Luciano:Norbert:Sarcone:M:1962-12-14:73069:909-96-1669
Newton:Jose:Piacente:M:1969-11-05:264422:967-61-8575
Gretta:Jennefer:Dipinto:F:1921-10-28:118497:907-49-2510
Jenni:Janella:Mcquiston:F:1969-07-11:137418:932-55-7164
Malena:Apryl:Kings:F:1980-01-08:98204:934-48-2334
Willy:Russell:Merker:M:1967-10-10:41026:992-83-4829
Jami:Allen:Mulkerin:F:1958-08-08:236024:934-86-4126
Fernando:Lowell:Zebell:M:1955-07-03:246516:951-57-6196
Garnet:Helaine:Edghill:F:1985-05-15:97125:979-42-4409
Siu:Cherrie:Lechelt:F:2012-07-24:148331:906-85-3202
Randal:Osvaldo:Torma:M:2013-12-17:275086:982-68-6906
Myles:Stuart:Miskell:M:1928-05-17:280726:992-89-2715
Conrad:Donnie:Piatkowski:M:1920-11-17:59935:916-33-7831
Darcel:Fleta:Czyrnik:F:1994-09-05:99641:995-95-8909
Juliane:Hallie:Vuillemot:F:1972-01-11:203415:950-17-2657
Cordia:Estella:Scheiber:F:1924-03-13:214853:965-74-4739
Bryon:Isiah:Nazario:M:2002-02-26:201478:963-38-1755
Alex:Gilbert:Trigueros:M:1990-12-30:271591:999-60-3232
Irwin:Rene:Zalesky:M:1931-06-20:122599:974-53-6387
Deandre:Brent:Bailor:M:1954-08-07:93429:920-78-9780
Cheree:Dorethea:Anspaugh:F:1985-01-17:278860:961-36-6578
Zachariah:Harris:Reemer:M:1924-12-16:238892:944-32-3013
See:Sharen:Howryla:F:1979-12-30:169570:925-12-1644
Mozella:Necole:Giannakopoulo:F:1993-08-24:40880:980-93-4844
David:Tracy:Delatrinidad:M:1932-01-26:85003:993-37-2066
Michel:Loren:Sandoz:M:1925-05-04:270046:948-56-4284
Graham:Denny:Apresa:M:1966-07-14:209118:928-67-3592
Kattie:Sammie:Ercek:F:2002-07-26:211993:996-32-1564
Shaunna:Kathlene:Amunrud:F:1965-07-01:234090:916-53-6216
Leeann:Madeleine:Clater:F:1976-10-26:100705:946-41-7003
Garry:Lino:Galm:M:1934-08-14:168769:994-29-1678
Shelly:Marhta:Krenzke:F:1915-05-08:270492:967-71-8565
Shantelle:Zoe:Spicher:F:1952-12-11:183419:989-79-8274
Yessenia:Lucina:Fred:F:1967-04-21:131376:990-95-7077
Del:Maynard:Dzurnak:M:1960-01-31:276319:938-46-9967
Maximina:Kirsten:Barranco:F:2004-11-05:257482:938-70-3631
Shasta:Bette:Licausi:F:2006-05-26:273311:929-84-1247
Wendell:Bud:Landi:M:2012-03-06:61215:987-95-9502
Sabine:Petronila:Lambertson:F:1937-03-31:274044:916-95-8546
Leonard:Filiberto:Sellberg:M:1982-05-25:264070:965-65-6038
Demetrice:Flossie:Schehr:F:1958-03-16:245090:990-42-7824
Reyes:Gilberto:Schlabs:M:1997-07-11:252679:906-83-6847
Bernard:Reggie:Coache:M:1960-06-23:53020:941-56-6401
Lavette:Elouise:Riedinger:F:1950-11-21:51371:913-81-9468
Edmund:Nicolas:Majocka:M:1974-05-26:298673:921-55-7816
Deana:Marcie:Malle:F:2011-06-24:15388:973-52-9173
Claudio:Leopoldo:Ealand:M:1916-01-14:158319:929-61-6825
Lucretia:Divina:Parnell:F:2011-05-30:41054:963-43-3135
Myrtis:Yaeko:Maciarello:F:1941-11-16:191903:996-58-7685
Cordie:Cara:Sheilds:F:2007-02-08:219449:950-98-5411
Michell:Corinne:Himmelspach:F:1968-02-27:154658:905-67-9613
Major:Shawn:Garay:M:1934-12-18:79152:904-49-5993
Andrew:Darnell:Kniceley:M:1966-09-02:25955:987-26-4880
Shasta:Floria:Messineo:F:2008-02-23:30339:925-83-2833
Madalene:Arline:Vanholland:F:1945-05-14:47734:977-68-6192
Clare:So:Bieker:F:1922-04-07:14603:922-40-9560
Audrey:Lorrine:Sprewell:F:1932-10-25:283164:997-53-7925
ARLEN:HAYDEN:CARVILL:M:1986-05-24:66754:918339442
Maybelle:Honey:Jolicoeur:F:1923-02-10:239772:902-46-1282
Darwin:Cedric:Gompf:M:1954-03-19:152991:926-88-9050
Jonelle:Maryam:Berkstresser:F:1944-11-17:253619:935-62-7974
Lynsey:Marcelina:Dagrella:F:1969-03-01:209989:978-42-7296
Robert:Jessie:Anaya:M:1922-09-30:136381:956-53-6

wasbs://training@dbtraincentralindia.blob.core.windows.net//dataframes/people-with-dups.txt

In [0]:
initialDF = (spark.read
   .option("header", "true")
    .option("inferSchema", "true")
    .option("delimiter", ":")
    .csv(sourceFile)
            )
# materialize the cache
initialDF.count()
print(initialDF)

DataFrame[firstName: string, middleName: string, lastName: string, gender: string, birthDate: timestamp, salary: int, ssn: string]

In [0]:
modifiedDF = (initialDF
  .select(
      upper(col("firstName")).alias("modFirstName"),
      upper(col("middleName")).alias("modMiddleName"),
      upper(col("lastName")).alias("modLastName"),
      col("gender"),col("birthDate"),col("salary"),
      translate(col("ssn"), "-", "").alias("ssnNumber")
    
   )
  .dropDuplicates(["modFirstName","modMiddleName","modLastName","gender","birthDate","salary","ssnNumber"])
 
  
)
print(modifiedDF.count())


100000

In [0]:
print(modifiedDF)
display(modifiedDF)


DataFrame[modFirstName: string, modMiddleName: string, modLastName: string, gender: string, birthDate: timestamp, salary: int, ssnNumber: string]

modFirstName,modMiddleName,modLastName,gender,birthDate,salary,ssnNumber
ELOISA,RUBYE,CAYOUETTE,F,2000-06-20T00:00:00.000+0000,204031,935899009
ANTIONE,RANDY,HAMACHER,M,2004-03-05T00:00:00.000+0000,271486,917963554
MALENA,APRYL,KINGS,F,1980-01-08T00:00:00.000+0000,98204,934482334
FERNANDO,LOWELL,ZEBELL,M,1955-07-03T00:00:00.000+0000,246516,951576196
MYLES,STUART,MISKELL,M,1928-05-17T00:00:00.000+0000,280726,992892715
JULIANE,HALLIE,VUILLEMOT,F,1972-01-11T00:00:00.000+0000,203415,950172657
ALEX,GILBERT,TRIGUEROS,M,1990-12-30T00:00:00.000+0000,271591,999603232
MICHEL,LOREN,SANDOZ,M,1925-05-04T00:00:00.000+0000,270046,948564284
SHAUNNA,KATHLENE,AMUNRUD,F,1965-07-01T00:00:00.000+0000,234090,916536216
LEEANN,MADELEINE,CLATER,F,1976-10-26T00:00:00.000+0000,100705,946417003


In [0]:
finalDF= (modifiedDF.toDF("firstName","middleName","lastName","gender","birthDate","salary","ssn")

         )



In [0]:
finalDF.printSchema()
display(finalDF)
print(finalDF.count())



root
-- firstName: string (nullable = true)
-- middleName: string (nullable = true)
-- lastName: string (nullable = true)
-- gender: string (nullable = true)
-- birthDate: timestamp (nullable = true)
-- salary: integer (nullable = true)
-- ssn: string (nullable = true)

firstName,middleName,lastName,gender,birthDate,salary,ssn
ELOISA,RUBYE,CAYOUETTE,F,2000-06-20T00:00:00.000+0000,204031,935899009
ANTIONE,RANDY,HAMACHER,M,2004-03-05T00:00:00.000+0000,271486,917963554
MALENA,APRYL,KINGS,F,1980-01-08T00:00:00.000+0000,98204,934482334
FERNANDO,LOWELL,ZEBELL,M,1955-07-03T00:00:00.000+0000,246516,951576196
MYLES,STUART,MISKELL,M,1928-05-17T00:00:00.000+0000,280726,992892715
JULIANE,HALLIE,VUILLEMOT,F,1972-01-11T00:00:00.000+0000,203415,950172657
ALEX,GILBERT,TRIGUEROS,M,1990-12-30T00:00:00.000+0000,271591,999603232
MICHEL,LOREN,SANDOZ,M,1925-05-04T00:00:00.000+0000,270046,948564284
SHAUNNA,KATHLENE,AMUNRUD,F,1965-07-01T00:00:00.000+0000,234090,916536216
LEEANN,MADELEINE,CLATER,F,1976-10-26T00:00:00.000+0000,100705,946417003


100000

In [0]:
(finalDF.write
   .mode("overwrite")
   .parquet(destFile)
)
finalDF = spark.read.parquet(destFile)
print("Total Records: {0:,}".format( finalDF.count() ))

Total Records: 100,000

In [0]:
display( dbutils.fs.ls(destFile) )

path,name,size,modificationTime
dbfs:/user/farzanam24@gmail.com/people.parquet/_SUCCESS,_SUCCESS,0,1662699283000
dbfs:/user/farzanam24@gmail.com/people.parquet/_committed_6453941352764611838,_committed_6453941352764611838,424,1662699282000
dbfs:/user/farzanam24@gmail.com/people.parquet/_started_6453941352764611838,_started_6453941352764611838,0,1662699282000
dbfs:/user/farzanam24@gmail.com/people.parquet/part-00000-tid-6453941352764611838-a213f959-2405-4290-91ea-66dc4e0f861d-925-1-c000.snappy.parquet,part-00000-tid-6453941352764611838-a213f959-2405-4290-91ea-66dc4e0f861d-925-1-c000.snappy.parquet,771223,1662699282000
dbfs:/user/farzanam24@gmail.com/people.parquet/part-00001-tid-6453941352764611838-a213f959-2405-4290-91ea-66dc4e0f861d-926-1-c000.snappy.parquet,part-00001-tid-6453941352764611838-a213f959-2405-4290-91ea-66dc4e0f861d-926-1-c000.snappy.parquet,771487,1662699282000
dbfs:/user/farzanam24@gmail.com/people.parquet/part-00002-tid-6453941352764611838-a213f959-2405-4290-91ea-66dc4e0f861d-927-1-c000.snappy.parquet,part-00002-tid-6453941352764611838-a213f959-2405-4290-91ea-66dc4e0f861d-927-1-c000.snappy.parquet,768272,1662699282000
dbfs:/user/farzanam24@gmail.com/people.parquet/part-00003-tid-6453941352764611838-a213f959-2405-4290-91ea-66dc4e0f861d-928-1-c000.snappy.parquet,part-00003-tid-6453941352764611838-a213f959-2405-4290-91ea-66dc4e0f861d-928-1-c000.snappy.parquet,764372,1662699282000


##![Spark Logo Tiny](https://s3-us-west-2.amazonaws.com/curriculum-release/images/105/logo_spark_tiny.png) Validate Your Answer

At the bare minimum, we can verify that you wrote the parquet file out to **destFile** and that you have the right number of records.

Running the following cell to confirm your result:

In [0]:
finalDF = spark.read.parquet(destFile)
finalCount = finalDF.count()

clearYourResults()
validateYourAnswer("01 Expected 100000 Records", 972882115, finalCount)
summarizeYourResults()
